In [1]:
# import the necessary packages
# from keras.applications import ResNet50
# todo: without these models so far
# from keras.applications import InceptionV3
# from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
# from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers import Dropout
import numpy as np
import argparse
import cv2
import h5py
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
# keras.clear_session()

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [3]:
# define a dictionary that maps model names to their classes
# inside Keras
MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
#     todo without these models so far
#     "inception": InceptionV3,
#     "xception": Xception, # TensorFlow ONLY
    "resnet": ResNet50
}

In [4]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

In [5]:
chosen_model = "vgg16"

In [6]:
# print("[INFO] loading {}...".format(chosen_model))
# Network = MODELS[chosen_model]
# model = Network(weights="imagenet")

[INFO] loading vgg16...


In [2]:
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.layers import Dense
import pandas as pd

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
from keras.applications.vgg16 import decode_predictions
from scipy.misc import imresize
import os

Using TensorFlow backend.


In [3]:
labels_path = os.getcwd() + "\\" + os.path.join( "data", "labels.csv", "")

In [4]:
labels_path

'C:\\Users\\Vladamir Mazin\\DeepNN\\data\\labels.csv\\'

In [5]:
data_labels = pd.read_csv(labels_path + "labels.csv")

In [6]:
type(data_labels)

pandas.core.frame.DataFrame

In [7]:
df = pd.DataFrame(np.random.randn(100, 2))
# df

In [8]:
def get_train_test(data_labels):
    msk = np.random.rand(len(data_labels)) < 0.8
    train = data_labels[msk]
    test = data_labels[~msk]
    return train, test

In [9]:
train, test = get_train_test(data_labels)

In [10]:
# train['id']

In [11]:
img_path = os.getcwd() + "\\" + os.path.join( "data", "train", "train", "")

In [12]:
# from itertools import islice
# for x in islice(train.iterrows(), 10):
#     print(x[1]['id'])

In [ ]:
# preparing the train dataset
train_img = []
for row in train.iterrows():
    temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
    temp_img = image.img_to_array(temp_img)
    train_img.append(temp_img)
    
#converting train images to array and applying mean subtraction processing
train_img = np.array(train_img) 
train_img = preprocess_input(train_img)

In [19]:
# applying the same procedure with the test dataset
test_img=[]
for row in test.iterrows():
    temp_img = image.load_img(img_path + row[1]['id'] + '.jpg', target_size=(224,224))
    temp_img = image.img_to_array(temp_img)
    test_img.append(temp_img)

test_img=np.array(test_img) 
test_img=preprocess_input(test_img)

In [50]:
# loading VGG16 model weights
model = VGG16(weights='imagenet', include_top=False)

# Extracting features from the train dataset using the VGG16 pre-trained model
features_train = model.predict(train_img)

# Extracting features from the test dataset using the VGG16 pre-trained model
features_test=model.predict(test_img)

InternalError: Dst tensor is not initialized.
	 [[Node: _arg_input_1_0_0/_157 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_126__arg_input_1_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: block5_pool/MaxPool/_159 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_127_block5_pool/MaxPool", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [49]:
# del model
# keras.backend.clear_session()

In [21]:
features_train.shape

(8182, 7, 7, 512)

In [23]:
# flattening the layers to conform to MLP input
# todo: check exact shapes
# train_x = features_train.reshape(25088, 8175)
train_x = features_train.reshape(8182, 25088)

In [24]:
# converting target variable to array
train_y = np.asarray(train['breed'])

In [25]:
# performing one-hot encoding for the target variable
train_y = pd.get_dummies(train_y)
train_y = np.array(train_y)

In [26]:
train_y.shape

(8182, 120)

In [27]:
# creating training and validation set
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [42]:
# creating a mlp model
from keras.layers import Dense, Activation
model=Sequential()

model.add(Dense(5000, input_dim=25088, activation='relu',kernel_initializer='uniform'))
model.add(Dropout(0.3))

model.add(Dense(1000,input_dim=5000,activation='sigmoid'))
model.add(Dropout(0.4))

model.add(Dense(500,input_dim=1000,activation='sigmoid'))
model.add(Dropout(0.2))

model.add(Dense(units=120))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [43]:
# fitting the model
model.fit(X_train, Y_train, epochs=20, batch_size=128,validation_data=(X_valid,Y_valid))

Train on 6545 samples, validate on 1637 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[25088,5000]
	 [[Node: dense_22/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=8501689, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_22/random_uniform/shape)]]

Caused by op 'dense_22/random_uniform/RandomUniform', defined at:
  File "c:\users\vladamir mazin\appdata\local\conda\conda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\vladamir mazin\appdata\local\conda\conda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\vladamir mazin\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\vladamir mazin\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\vladam

#### batch_size = 32

In [146]:
# fitting the model
# model.fit(X_train, Y_train, epochs=20, batch_size=32,validation_data=(X_valid,Y_valid))